<a href="https://colab.research.google.com/github/Krishnabhuvan/machinelearning-feature-engineering/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

In [ ]:
df.drop(columns=['PassengerId','Name' , 'Ticket', 'Cabin'],inplace=True)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)
#

In [ ]:
X_train.head()

In [ ]:
y_train.sample(5)

In [ ]:
trf1 = ColumnTransformer(
    [('impute_age',SimpleImputer(),[2]),('impute_embared',SimpleImputer(strategy='most_frequent'),[6])],
    remainder='passthrough'
)

In [ ]:
trf2 = ColumnTransformer([
        ('ohe_sex_embarked',OneHotEncoder(handle_unknown='ignore', sparse_output=False),[1,6])
 ] ,remainder='passthrough')

In [ ]:
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [ ]:
trf4 = SelectKBest(score_func=chi2,k=8)

In [ ]:
trf5 = DecisionTreeClassifier()

In [ ]:
#pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)   # now step name is exactly 'trf5'
])

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

In [ ]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_


In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
import numpy as np


In [ ]:
pipe = pickle.load(open('/content/pipe.pkl','rb'))

In [ ]:
test_input2 = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)


In [ ]:
pipe.predict(test_input2)
